In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [3]:
df.shape

(250, 38)

In [4]:
df = df[['Title', 'Genre', 'Director', 'Actors', 'Plot']]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [5]:
df['Actors'] = df['Actors'].map(lambda x: x.lower().split(','))

In [6]:
df['Genre'] = df['Genre'].map(lambda x: x.lower().split(','))

In [7]:
df['Director'] = df['Director'].map(lambda x: x.lower().split(' '))

In [8]:
for index, row in df.iterrows():
    row['Genre'] = [x.lower().replace(' ', '') for x in row['Genre']]
    row['Actors'] = [x.lower().replace(' ', '') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()

In [9]:
df['Genre'].head()

0            [crime, drama]
1            [crime, drama]
2            [crime, drama]
3    [action, crime, drama]
4            [crime, drama]
Name: Genre, dtype: object

In [10]:
df['Actors'].head()

0    [timrobbins, morganfreeman, bobgunton, william...
1    [marlonbrando, alpacino, jamescaan, richards.c...
2    [alpacino, robertduvall, dianekeaton, robertde...
3    [christianbale, heathledger, aaroneckhart, mic...
4    [martinbalsam, johnfiedler, leej.cobb, e.g.mar...
Name: Actors, dtype: object

In [11]:
df['Director'].head()

0         frankdarabont
1    francisfordcoppola
2    francisfordcoppola
3      christophernolan
4           sidneylumet
Name: Director, dtype: object

In [12]:
import rake_nltk
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
df['Key_words'] = ''

for index, row in df.iterrows():
    plot = row['Plot']
    r = Rake()
    r.extract_keywords_from_text(plot)
    
    key_words_dict_scores = r.get_word_degrees()
    row['Key_words'] = list(key_words_dict_scores.keys())
    
df = df.drop(columns=['Plot'])

In [14]:
df.head()['Key_words'][0]

['two',
 'imprisoned',
 'men',
 'bond',
 'number',
 'years',
 'finding',
 'solace',
 'eventual',
 'redemption',
 'acts',
 'common',
 'decency']

In [15]:
df.set_index('Title', inplace=True)
df.head()

,Genre,Director,Actors,Key_words
Title,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton, william...","[two, imprisoned, men, bond, number, years, fi..."
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan, richards.c...","[aging, patriarch, organized, crime, dynasty, ..."
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton, robertde...","[early, life, career, vito, corleone, 1920s, n..."
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart, mic...","[menace, known, joker, emerges, mysterious, pa..."
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb, e.g.mar...","[jury, holdout, attempts, prevent, miscarriage..."


In [16]:
df['bag_of_words'] = ''
columns = df.columns

for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words = words + ' '.join(row[col]) + ' '
        else:
            words = words + row[col] + ' '
    row['bag_of_words'] = words
    
df.drop(columns=[col for col in df.columns if col != 'bag_of_words'], inplace=True)

In [17]:
df.head()

,bag_of_words
Title,
The Shawshank Redemption,crime drama frankdarabont timrobbins morganfre...
The Godfather,crime drama francisfordcoppola marlonbrando al...
The Godfather: Part II,crime drama francisfordcoppola alpacino robert...
The Dark Knight,action crime drama christophernolan christianb...
12 Angry Men,crime drama sidneylumet martinbalsam johnfiedl...


In [18]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

In [19]:
c = count_matrix.todense()

In [21]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.14638501, 0.1315587 , ..., 0.05      , 0.05      ,
        0.05270463],
       [0.14638501, 1.        , 0.34236839, ..., 0.048795  , 0.048795  ,
        0.05143445],
       [0.1315587 , 0.34236839, 1.        , ..., 0.0438529 , 0.0438529 ,
        0.04622502],
       ...,
       [0.05      , 0.048795  , 0.0438529 , ..., 1.        , 0.05      ,
        0.05270463],
       [0.05      , 0.048795  , 0.0438529 , ..., 0.05      , 1.        ,
        0.05270463],
       [0.05270463, 0.05143445, 0.04622502, ..., 0.05270463, 0.05270463,
        1.        ]])

In [22]:
indices = pd.Series(df.index)

In [23]:
def recommendations(title, cosine_sim = cosine_sim):
    recommended_movies = []
    
    idx = indices[indices == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)
    top_10_movies = list(score_series.iloc[1:11].index)
    print(top_10_movies)
    
    for i in top_10_movies:
        recommended_movies.append(list(df.index)[i])
    return recommended_movies

In [27]:
recommendations('The Godfather')

[2, 83, 128, 226, 100, 15, 76, 123, 110, 66]


['The Godfather: Part II',
 'Scarface',
 'Fargo',
 'Rope',
 'On the Waterfront',
 'Goodfellas',
 'Baby Driver',
 'Cool Hand Luke',
 'Casino',
 'A Clockwork Orange']